In [4]:
import pandas as pd
import altair as alt
import ipywidgets as widgets
from IPython.display import display, clear_output

df = pd.read_csv('https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/building_inventory.csv')

df = df[df['City'].isin(['Urbana', 'Champaign'])]

In [ ]:
# year ranges 

year_ranges = []
for start in range(1870, 2020, 10):
    end = start + 10
    year_ranges.append((f'{start}-{end-1}', (start, end)))

range_widget = widgets.Dropdown(
    options=[("Select a range", None)] + year_ranges,
    description='Year Range:'
)

# color scheme 

color_schemes = [ "darkred","darkgold", "darkgreen", "plasma", "inferno","magma"]
color_widget = widgets.Dropdown(
    options=["Select a color scheme"] + color_schemes,
    description='Color Scheme:'
)


output = widgets.Output()

def update_chart(year_range, color_scheme):
    with output:
        clear_output()
        if year_range is None or color_scheme == "Select a color scheme":
            display("Select a decade and color mapping scheme of your choice")
            return # break 
        
        # continue 
        start, end = year_range
        df_filtered = df[(df['Year Constructed'] >= start) & (df['Year Constructed'] < end)]
        
        bars = df_filtered.groupby(['Year Constructed', 'Usage Description']).size().reset_index(name='Count')

        chart = alt.Chart(bars).mark_bar().encode(
            x=alt.X('Year Constructed:O', title='Year', axis=alt.Axis(labelAngle=-45)),
            y=alt.Y('sum(Count):Q', title='Total Buildings'),
            color=alt.Color('Usage Description:N', scale=alt.Scale(scheme=color_scheme), title='Use Description'),
            tooltip=['Year Constructed:O', 'sum(Count):Q', 'Usage Description:N']
        ).properties(
            width=600,
            height=400,
            title=f'Buildings Constructed In U-C Area By Year Constructed And Their Use {start}-{end-1}'
        )
        
        display(chart)
        chart.save('../assets/json/line_chart.json')
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        update_chart(range_widget.value, color_widget.value)




range_widget.observe(on_change)
color_widget.observe(on_change)


display(range_widget)
display(color_widget)
display(output)


with output:
    display("Select a decade and color mapping scheme of your choice")
    


Dropdown(description='Year Range:', options=(('Select a range', None), ('1870-1879', (1870, 1880)), ('1880-188…

Dropdown(description='Color Scheme:', options=('Select a color scheme', 'darkred', 'darkgold', 'darkgreen', 'p…

Output()

In [8]:

df['Decade'] = (df['Year Constructed'] // 10) * 10

agg_data = df.groupby('Decade')['Square Footage'].sum().reset_index()

line_plot = alt.Chart(agg_data).mark_line(point=True, strokeWidth=3).encode(
    x=alt.X('Decade:O', title='Decade', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('Square Footage:Q', title='Total Square Footage'),
    tooltip=['Decade:O', 'Square Footage:Q']
).properties(
    width=600,
    height=400,
    title='Total Square Footage of Buildings by Decade'
) + alt.Chart(agg_data).mark_area(opacity=0.3, color='blue').encode(
    x=alt.X('Decade:O'),
    y=alt.Y('Square Footage:Q')
)

line_plot.display()
line_plot.save('../assets/json/line_chart.json')

alt.LayerChart(...)